<a href="https://colab.research.google.com/github/yoavdana/computational-physics-4/blob/main/comp_phy4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

computational physics EX4 -Yoav Dana

In [15]:
import numpy as np
from matplotlib import pyplot as plt



part 1: jacobian numerical calculation and NR root solver

In [16]:
def numerical_jacobian(f, xs, dx=1e-8):

    ys = f(xs)

    x_dim = xs.shape[0]
    y_dim = ys.shape[0]

    jac = np.empty((y_dim, x_dim))

    for i in range(x_dim):
        part_der_element = e(x_dim, i + 1)
        x_plus = xs + dx * part_der_element
        x_minus = xs - dx * part_der_element
        jac[:, i] = (f(x_plus) - f(x_minus)) / (2 * dx)

    return jac


def e(n, i):
  ret = np.zeros(n)
  ret[i - 1] = 1.0
  return ret

Jacobian calculation test: i choose a few functions with a known jacobian and tested the function "numerical_jacobian"

In [17]:

def f1(x):

    f1 =x[0]
    f2 =5*x[2]
    f3=4*x[1]**2-2*x[2]
    f4=x[2]*np.sin(x[0])

    return np.array([f1,f2,f3,f4])

def real_jac1(x):
    return np.array([[1,0,0],[0,0,5],[0,8*x[1],-2],[x[2]*np.cos(x[0]),0,np.sin(x[0])]])


def f2(x):

    f1 = x[0]**2-x[1]*x[2]
    f2 = x[0]*x[1]*x[2]

    return np.array([f1, f2])

def real_jac2(x):
    return np.array([[2*x[0],-x[2],-x[1]],[x[1]*x[2],x[0]*x[2],x[0]*x[1]]])


x = np.array([1,4,2], dtype=float)

print('numerical jacobian=',numerical_jacobian(f1, x, dx=1e-8))
print('real jacobian=',real_jac1(x))

print('numerical jacobian=',numerical_jacobian(f2, x, dx=1e-8))
print('real jacobian=',real_jac2(x))


numerical jacobian= [[ 1.          0.          0.        ]
 [ 0.          0.          5.00000006]
 [ 0.         31.99999981 -2.00000017]
 [ 1.08060462  0.          0.84147098]]
real jacobian= [[ 1.          0.          0.        ]
 [ 0.          0.          5.        ]
 [ 0.         32.         -2.        ]
 [ 1.08060461  0.          0.84147098]]
numerical jacobian= [[ 1.99999999 -1.99999999 -3.99999998]
 [ 8.          1.99999999  3.99999998]]
real jacobian= [[ 2. -2. -4.]
 [ 8.  2.  4.]]
